In [ ]:
import requests
import json
import time
import pandas as pd
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Configuración para Qwen3-8B
BASE_URL = "http://localhost:8001"
MODEL_NAME = "qwen3-8b-awq"

print("🚀 QWEN3-8B-AWQ FILATELIA ASSISTANT")
print("="*60)
print(f"🔗 URL: {BASE_URL}")
print(f"🤖 Modelo: {MODEL_NAME}")
print(f"🎯 Propósito: Resumir y analizar documentos filatélicos")
print("="*60)

In [ ]:
def verificar_qwen3():
    """Verifica que Qwen3-8B esté corriendo"""
    try:
        response = requests.get(f"{BASE_URL}/health", timeout=10)
        if response.status_code == 200:
            print("✅ Qwen3-8B está corriendo")
            return True
        else:
            print(f"❌ Error del servidor: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ No se puede conectar a Qwen3-8B")
        print("💡 Verifica que el contenedor esté corriendo:")
        print("   docker ps --filter 'name=vllm-qwen3'")
        return False
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return False

def obtener_info_modelo():
    """Obtiene información del modelo"""
    try:
        response = requests.get(f"{BASE_URL}/v1/models", timeout=10)
        if response.status_code == 200:
            models = response.json()
            print("📋 Información del modelo:")
            for model in models['data']:
                print(f"   🤖 ID: {model['id']}")
                print(f"   📅 Creado: {time.ctime(model.get('created', 0))}")
            return True
        else:
            print(f"❌ Error obteniendo info: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Verificación inicial
print("🔍 Verificando conexión con Qwen3-8B...")
servidor_ok = verificar_qwen3()

if servidor_ok:
    obtener_info_modelo()
else:
    print("\n🛠️ COMANDOS ÚTILES:")
    print("   Ver contenedor: docker ps")
    print("   Ver logs: docker logs vllm-qwen3")
    print("   Iniciar: docker start vllm-qwen3")

In [ ]:
def resumir_documento_filatelico(texto, tipo_resumen="completo", max_tokens=800):
    """
    Resume documentos filatélicos con Qwen3-8B
    
    Args:
        texto: Texto a resumir
        tipo_resumen: "completo", "tecnico", "historico", "breve"
        max_tokens: Máximo de tokens en la respuesta
    """
    
    prompts = {
        "completo": f"""
Analyze this philatelic document and create a structured summary. Do NOT invent or add information.

HISTORICAL: Key dates, events, periods, relevant figures  
TECHNICAL: Paper, perforation, printing, colors, dimensions, etc.  
ISSUE: Denominations, quantities, series, printers, etc.  
PHILATELIC: Rarity, market value, known varieties, etc.  
ERRORS: Inverted centers, double prints, missing colors, shifted perforations, incorrect overprints, misregistrations, broken frames, etc.  

IMPORTANT: Include only the categories with real information present in the document. If a category has no data, omit it entirely.  

Also include any historical prices, condition/state of preservation, and technical details explicitly mentioned.  

Document:  
{texto}  

Summary:
""",

        "tecnico": f"""Como especialista en aspectos técnicos filatélicos, analiza este documento enfocándote en:

- Métodos de impresión y papeles utilizados
- Dentados y perforaciones
- Variantes de color y errores de impresión
- Marcas de agua y elementos de seguridad
- Calidad y conservación

Documento:
{texto}

Análisis técnico:""",

        "historico": f"""Como historiador de la filatelia costarricense, extrae y analiza:

- Contexto histórico de las emisiones
- Eventos conmemorados
- Figuras históricas representadas
- Evolución temporal de las series
- Impacto cultural y social

Documento:
{texto}

Análisis histórico:""",

        "breve": f"""Resume este documento filatélico en máximo 3 párrafos, manteniendo solo la información más importante:

{texto}

Resumen breve:"""
    }
    
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "system",
                "content": "Eres un experto filatélico especializado en Costa Rica con amplio conocimiento en historia, técnicas de impresión y valoración de estampillas."
            },
            {
                "role": "user",
                "content": f"/no_think {prompts.get(tipo_resumen, prompts['completo'])}"
            }
        ],
        "max_tokens": max_tokens,
        "temperature": 0.3,  # Más determinístico para información factual
        "top_p": 0.9,
        "enable_thinking": False  # Directamente en el payload

    }
    
    try:
        print(f"🤖 Generando resumen {tipo_resumen} con Qwen3-8B...")
        start_time = time.time()
        
        response = requests.post(
            f"{BASE_URL}/v1/chat/completions",
            json=payload,
            timeout=120  # Más tiempo para modelo grande
        )
        
        generation_time = time.time() - start_time
        
        if response.status_code == 200:
            result = response.json()
            resumen = result['choices'][0]['message']['content']
            usage = result.get('usage', {})
            
            # Mostrar resultados
            print(f"✅ Resumen generado en {generation_time:.2f} segundos")
            print(f"📊 Tokens: {usage.get('prompt_tokens', 0)} → {usage.get('completion_tokens', 0)} (Total: {usage.get('total_tokens', 0)})")
            
            if usage.get('total_tokens', 0) > 0:
                velocidad = usage.get('total_tokens', 0) / generation_time
                print(f"🚀 Velocidad: ~{velocidad:.1f} tokens/segundo")
            
            return {
                'tipo': tipo_resumen,
                'resumen': resumen,
                'tiempo': generation_time,
                'tokens': usage.get('total_tokens', 0),
                'velocidad': velocidad if 'velocidad' in locals() else 0
            }
        else:
            print(f"❌ Error HTTP: {response.status_code}")
            print(f"Respuesta: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error generando resumen: {e}")
        return None

print("✅ Función de resumen filatélico lista")

In [ ]:
# Textos de ejemplo sobre filatelia costarricense
textos_filatelicos = {
    "ejemplo_1": """
## PARTE SEGUNDA — LA PLANCHA

La plancha usada para cancelar con las barras los estampillas enumeradas en pógine precedente, se componía de 5 grupos de barras paralelas de 5 lineas cada uno. Es posible que el método de impresión fuera el de imprimir la mítad de un pliego completo de 100 estampillas y posteriormente se dio vuelta al pliego en el piano horizontal para imprimir la mítad restante. Una explicitación más lógica se opoya en la teoria sustentada por el ing. Perolta: los pliegos doblados fueron insertedos en la prensa y luego se voltearon. En cualquier caso el resultado es el mismo, la mitad inferior siempre presentándose invertida en la parte superior.

Varios puntos de vista dan fundamento a la teoria de los pliegos doblados:

1)—Es más fácil dar vuelta a un pliego

doblado que manipular los pliegos com plefos.

2)—Con toda probabilidad, cuondo menos los “telégrafos” y los provisionales de “1911” ya se encontrolaban doblados, y habian sido sobrecargados con planches de 50 tipos.

3!—El método usual de almacener es tas estampillas era doblándolas con un pa pel encerado separador entre cada pliego y parece altamente ilógico que pudieser haber sido desdobladas para cancelaría con las barras.

. 4)—Oiro fuerte punto de vista es que muchos de estos “barrados” que todavia retienen su gomo original, conservan trazos de la impresión de las barras de estampilla opvestas, igual que el un colón “albino”, que aparece al reverso en relieve.

""",
"ejemplo_2" : """
## From Hind to Woo Inverted Center 2c stamp of 19011

Fred O'Neill

The inverted center stamp 2c of 1901 is the only known copy. This singular stamp is mint with perforation 15 of the same normal color of the regular issued which was engraved and printed in sheets of 100 by Waterlow & Sons of London, a guaranty the seriousness and honesty of the printing work. Logically there should have been a sheet of 100 with this error but the fate of the other 99 stamps is unknown. It has gum exactly the same as the regular issue and it had not hinge marks indicating that it was never in a page of an album. In my opinion is that during the destruction of defective material a Waterlow employee separated it from the rest of the sheet and sold it in the philatelic market.

This stamp was unknown until it appeared in the auction of the famous collection of Arthur Hind in 1935. Hind was a famous collector of the 1910-30 and probably the last to collect the entire world. At his death in 1933 the world was going through an economic crisis and it was not the best time to sell a stamp collection. The sale started in 1933 but because of the poor early re-

results it was stopped. Later a British group liquidated the balance of the collection in 1935. The proceeds were $700,000 which was less than 60% of \\ the cost to acquire the collection. In \\ one of the auction sessions by H.R. \\ Harmer the 2c inverted center stamp \\ showed up and it was sold for 12 \\ pounds sterling. The buyer of the \\ stamp and other Costa Rica material \\ was Alex A. Cohen. His name was \\ not made public at that time because \\ of his position as attaché and general \\ consul of the American Embassy in \\ Costa Rica. The stamp was sold to \\ Carlos Echeverria Carazo who kept in \\ his collection until 1945. The stamp \\ was gifted to Alvaro Bonilla Lara. \\ The Bonilla Lara collection was sold \\ in auction again by H.R. Harmer on \\ Mary 1st and 2nd. of 1961 and this \\ stamp was listed as lot 149 without \\ reserve and sold for $550.00 to New York collector Norman Hubbard.

circa 1923 also owned the One Penny British Guinea of 1856.
""",
"ejemplo_3": """
## The Costa Rica 1881-82 and the 1883 UPU Surcharge Forgeries/Fantasies (2015). It is a 52 page monograph by Raul Hernandez showing in detail the material described in the title. It has been serialized in recent Oxcart so it will not require further introduction. The booklet is in color in high quality laser paper. Price $20.00 plus postage, order it from the author.

The Costa Rica

1881-82 and the 1883 U.P.U

Surcharge Forgeries/Fantasies.

Raul F. Hernandez

First Edition

![Figure](figures/OXCART222_page_013_figure_009.png)

2015

<table><tr><td></td><td>PLAN DE EMISIONES 2016*</td><td>CORREOS</td></tr><tr><td>1</td><td>200 años del nacimiento de Francisca Carrasco</td><td>4 de abril</td></tr><tr><td>2</td><td>100 años del nacimiento de José Joaquín Trejos</td><td>18 de abril</td></tr><tr><td>3</td><td>Cultura Cantonal: Teatro Minerva</td><td>3 de junio</td></tr><tr><td>4</td><td>25 Aniversario Museo Calderón Guardia</td><td>17 de junio</td></tr><tr><td>5</td><td>Siños Arqueológicos</td><td>23 junio</td></tr><tr><td>6</td><td>América UPAEP 2016: Juegos Olímpicos</td><td>30 de junio</td></tr><tr><td>7</td><td>Parques Nacionales 2016: Marino Las Baulas</td><td>24 de agosto</td></tr><tr><td>8</td><td>Escritores Nacionales: Yolando Oreamuno y Aquileo J. Echeverría</td><td>10 de octubre</td></tr><tr><td>9</td><td>Sobretasa Pro Ciudad de los Niños: Frutas</td><td>1 de diciembre</td></tr></table>
"""
}

print("📚 Textos filatélicos de prueba cargados:")
for clave, texto in textos_filatelicos.items():
    print(f"   📄 {clave}: {len(texto)} caracteres")

In [ ]:
resultados_resumenes = []

if servidor_ok:
    print("🔬 EJECUTANDO TESTS DE RESUMEN FILATÉLICO")
    print("="*60)
    
    # Test 1: Resumen completo del texto histórico
    print("\n📖 TEST 1: RESUMEN HISTÓRICO COMPLETO")
    print("-" * 40)
    resultado1 = resumir_documento_filatelico(
        textos_filatelicos["ejemplo_1"], 
        tipo_resumen="completo",
        max_tokens=500
    )
    
    if resultado1:
        resultados_resumenes.append(resultado1)
        print(f"\n📝 RESUMEN GENERADO:")
        print(resultado1['resumen'])  
    
    resultado2 = resumir_documento_filatelico(
        textos_filatelicos["ejemplo_2"], 
        tipo_resumen="completo",
        max_tokens=500
    )
    
    if resultado2:
        resultados_resumenes.append(resultado2)
        print(f"\n📝 RESUMEN GENERADO:")
        print(resultado2['resumen'])   
    
    resultado3 = resumir_documento_filatelico(
        textos_filatelicos["ejemplo_3"], 
        tipo_resumen="completo",
        max_tokens=500
    )
    
    if resultado3:
        resultados_resumenes.append(resultado3)
        print(f"\n📝 RESUMEN GENERADO:")
        print(resultado3['resumen'])   
    

else:
    print("⚠️ Saltando tests - Qwen3-8B no disponible")